In [19]:
import pickle
import json
import re
import nltk
from nltk.stem.porter import *

In [2]:
idf_pickle_file = '../../robust04_data/IDF.pkl'
idf = pickle.load( open(idf_pickle_file, "rb" ) )

In [3]:
type(idf)

dict

In [21]:
clean = lambda t: re.sub('[,?;*!%^&_+():-\[\]{}]', ' ', t.replace('"', ' ').replace('/', ' ').replace('\\', ' ').replace("'", ' ').replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('-', ' ').replace('.', '').replace('&hyph;', ' ').replace('&blank;', ' ').strip().lower())

# To tokenize a text field, first preprocess it using the "clean" lambda function, then tokenize preproceesed text with nltk tokenizer.

raw_text = '80-month'
preprocessed_text = clean(raw_text)
tokens = nltk.word_tokenize(preprocessed_text)

print('Raw text: {0}'.format(raw_text))
print('Preprocessed text: {0}'.format(preprocessed_text))
print('Tokens: {0}'.format(tokens))

Raw text: 80-month
Preprocessed text: 80 month
Tokens: ['80', 'month']


In [4]:
idf['patient']

4.926524998039284

In [37]:
with open('./ir_lmart_config.json') as j_file:
    lmart_config = json.load(j_file)
    

In [38]:
lmart_config

{'h_params': {'n_trees': {'max': '50', 'step': '1', 'min': '1'},
  'n_leaves': {'max': '50', 'step': '1', 'min': '1'},
  'shrinkage': {'max': '1', 'step': '0.1', 'min': '0.1'}},
 'budget': {'name': 'early_stop', 'max': '100', 'min': '10'}}

In [5]:
# i = 0
# for key, value in idf.items():
#     if i > 100:
#         break
#     print(key,value)
#     i += 1

In [6]:
unique_terms = idf.keys

In [7]:
dic_keys = [key for key in idf.keys()]
# dic_keys = dic_keys[9000000:10000000]
    

In [8]:
len(dic_keys)

1401234

In [9]:
dic_keys = sorted(dic_keys, key=str.lower)

In [10]:
len(dic_keys)

1401234

In [11]:
stemmer = PorterStemmer()

In [12]:
# stemmed = [stemmer.stem(word) for word in dic_keys]

In [13]:
stemmed = []

for word in dic_keys:
    if len(stemmed)%500000 == 0:
        print(word)
    stemmed.append(stemmer.stem(word))


#
careerist
over-looked


In [14]:
unique_stemmed = set(stemmed)
len(stemmed)

1401234

In [15]:
len(unique_stemmed)

1235800

In [16]:
for i,value in enumerate(unique_stemmed):
    if i > 100: break
    print(value)

408-727-6116
thurza
swensonsa
789420019
hochdeutsch
lien
§4301a-f
non-south
|afflower-se
archcliff
djazzara
purple-r
29466
elsberg
23260
433964
§401
40613b
khanka
oboroneksport
guarantee-schem
sedey
648-5035
four-lett
jasnogora
84532
§2101
05827
streetclark
1656p
tuten
bloodorgan
selfhood
all-compatriot
abu-shariah
walla
|mileag
219012580
4695m
fellow-profession
kbal
rose-johnson
mathal
a-|el
flra
fr29770
tsaritsino
petersburg-lvov
makshanoff
z113
pespi-cola
hasselt
stock-split
25099
al-tawq
95824
12511
7000-9000
heartguide-approv
pa0406032694
unzon
vrbovik
macenro
y715bn
211453
midsent
1150-square-foot
yorkpuerto
hyong-chin
010394b
chickelero
dubuch
enounc
reenforc
sikh
over-tir
events-monitor
brittenbroadhurst
ambarsumov
172627
1051receipt
122500
3592387
renaudin
reform1
ex-drive-in
tyoplyy
angioplastygreg
kankahi
§5902a2
less-regul
548-8755
islom
27477
80-month
4230stack
9211
fl3102
broll-tertian
368885
gloeckner


In [17]:
if 'ofmicrobes' in unique_stemmed:
    print('yes')

In [22]:
save_file = './original_vocab.txt'
with open(save_file, 'wt', encoding='utf-8') as voc_f:
     for i,item in enumerate(unique_stemmed):
        voc_f.write(item + '\n')